## Deliverable 2. Create a Customer Travel Destinations Map.

In [40]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [41]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,73.00,30,0,6.91,clear sky
1,1,Cape Town,ZA,-33.93,18.42,62.01,100,90,8.05,overcast clouds
2,2,Kozhevnikovo,RU,56.26,83.97,30.00,87,100,9.86,overcast clouds
3,3,Panalingaan,PH,8.78,117.42,81.07,80,100,4.74,overcast clouds
4,4,Airai,TL,-8.93,125.41,81.05,48,14,3.09,few clouds


In [42]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [43]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,73.00,30,0,6.91,clear sky
1,1,Cape Town,ZA,-33.93,18.42,62.01,100,90,8.05,overcast clouds
5,5,Saint George,US,37.10,-113.58,66.20,20,1,3.13,clear sky
6,6,Grado,ES,43.39,-6.07,59.00,58,20,10.29,few clouds
10,10,Fortuna,US,40.60,-124.16,53.60,62,1,10.29,clear sky
12,12,Salalah,OM,17.02,54.09,69.80,32,0,3.36,clear sky
13,13,Nizip,TR,37.01,37.79,55.40,50,0,3.04,clear sky
14,14,San Quintin,MX,30.48,-115.95,65.14,80,68,7.05,broken clouds
17,17,Port Lincoln,AU,-34.73,135.87,55.04,60,100,18.79,overcast clouds
18,18,Jamestown,US,42.10,-79.24,62.60,77,90,21.92,thunderstorm


In [44]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                353
City                   353
Country                347
Lat                    353
Lng                    353
Max Temp               353
Humidity               353
Cloudiness             353
Wind Speed             353
Current Description    353
dtype: int64

In [45]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
#preferred_cities_clean_df.notnull().sum()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,73.00,30,0,6.91,clear sky
1,1,Cape Town,ZA,-33.93,18.42,62.01,100,90,8.05,overcast clouds
5,5,Saint George,US,37.10,-113.58,66.20,20,1,3.13,clear sky
6,6,Grado,ES,43.39,-6.07,59.00,58,20,10.29,few clouds
10,10,Fortuna,US,40.60,-124.16,53.60,62,1,10.29,clear sky


In [46]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country","Max Temp","Current Description","Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,73.00,clear sky,-33.65,115.33,
1,Cape Town,ZA,62.01,overcast clouds,-33.93,18.42,
5,Saint George,US,66.20,clear sky,37.10,-113.58,
6,Grado,ES,59.00,few clouds,43.39,-6.07,
10,Fortuna,US,53.60,clear sky,40.60,-124.16,
12,Salalah,OM,69.80,clear sky,17.02,54.09,
13,Nizip,TR,55.40,clear sky,37.01,37.79,
14,San Quintin,MX,65.14,broken clouds,30.48,-115.95,
17,Port Lincoln,AU,55.04,overcast clouds,-34.73,135.87,
18,Jamestown,US,62.60,thunderstorm,42.10,-79.24,


In [47]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
       
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [48]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
#hotel_df.notnull().sum()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,73.00,clear sky,-33.65,115.33,Observatory Guest House
1,Cape Town,ZA,62.01,overcast clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
5,Saint George,US,66.20,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
6,Grado,ES,59.00,few clouds,43.39,-6.07,El reguero de Villamarín
10,Fortuna,US,53.60,clear sky,40.60,-124.16,Super 8 by Wyndham Fortuna


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined